# Модель предсказания срывов поставок на основе блендинга LightGBM, CatBoost, XGBoost

## Библиотеки и модули

Загрузка библиотек и модулей, используемых в решении. 
- Модель: Блендинг **LGBMClassifier**, **CatBoostClassifier**, **XGBoostClassifier** из библиотек **LightGBM**, **CatBoost**, **XGBoost**
- Предобработчик данных для модели: **DataPreprocessor** (из модуля **helper.data**)
- Борьба с дисбалансом классов: **RandomOverSampler** (библиотека **imbalanced-learn**)
- Разделение на тренировочную и тестовую выборки, кросс-валидация, оценка решения: 
    - библиотека **scikit-learn**
    - **valid_predictions** (из модуля **helpers.validation**)
- Работа с датасетом: библиотека **pandas** 
- Работа с файловой системой: модуль **os**

In [1]:
# Модель
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Пайплайн
from sklearn.pipeline import Pipeline
from helpers.data import DataPreprocessor, CAT_FEATURES
from helpers.validation import valid_predictions

# Данные
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

# Проверка GPU
from catboost.utils import get_gpu_device_count
TASK_TYPE = 'GPU' if get_gpu_device_count() else 'CPU'
TREE_METHOD = 'gpu_hist' if get_gpu_device_count() else 'auto'


# Настройки вывода
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Загрузка датасетов

In [ ]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')
SUBMISSION_PATH = os.path.join(ROOT, '../submissions/')

# Функции
def make_predictions(model: object | Pipeline, X_test: pd.DataFrame) -> None:
    """ Функция создания предсказаний для тестовой выборки.
    
    Обучает переданную модель на сбалансированном датасете, учитывая выбросы.
    Предсказания сохраняются с помощью функции save_submission.
    
    Параметры:
        model: Модель или пайплайн, используемый для обучения и предсказания меток
            тестовой выборки.
        X_test: Тестовая выборка, для которой будут сделаны предсказания.
            Экземпляр pandas.DataFrame."""
    
    # Загрузка сбалансированного датасета
    balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
    
    # Обрезка негативных записей до числа позитивных
    first_negatives = balanced_df[balanced_df['y'] == 0][:balanced_df[balanced_df['y'] == 1]['y'].count()]
    balanced_df = pd.concat([balanced_df[balanced_df['y'] == 1], first_negatives])

    # Удаление выбросов
    balanced_df = balanced_df[balanced_df['Длительность'] < 400]
    balanced_df = balanced_df[(balanced_df['Сумма'] > 2) & (balanced_df['Сумма'] < 10)]
    balanced_df = balanced_df[balanced_df['До поставки'] < 300]
    balanced_df = balanced_df[balanced_df['Дней между 0_1'] < 400]
    balanced_df = balanced_df[balanced_df['Количество изменений после согласований'] < 2000]
    balanced_df = balanced_df[balanced_df['Количество'] < 300000]
    
    # Разделение независимых и независимых переменных
    X, y = balanced_df.iloc[:, :-1], balanced_df.iloc[:, -1]   

    # Обучение модели и создание предсказаний для тестовой выборки
    model.fit(X, y)
    preds = model.predict(X_test)

    # Сохранение предсказаний
    save_submission(preds, 'submission')
    
def save_submission(preds: list | pd.DataFrame | pd.arrays.PandasArray, subname: str) -> None:
    subname = os.path.join(SUBMISSION_PATH, f'{subname}.csv')
    submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})
    submit_df.to_csv(subname, index=False)


# Загрузка датасетов
train_df = pd.read_csv(TRAIN_DATASET)
test_df = pd.read_csv(TEST_DATASET)

# Удаление дубликатов из тренировочной выборки
train_df = train_df.drop_duplicates()

# Разделение выборки на тренировочную и тестовую
X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Удаление выбросов из тренировочной выборки
X_train = pd.concat([X_train, y_train], axis=1) # Объединяем X_train и y_train
X_train = X_train[X_train['Длительность'] < 400]
X_train = X_train[(X_train['Сумма'] > 2) & (X_train['Сумма'] < 10)]
X_train = X_train[X_train['До поставки'] < 300]
X_train = X_train[X_train['Дней между 0_1'] < 400]
X_train = X_train[X_train['Количество изменений после согласований'] < 2000]
X_train = X_train[X_train['Количество'] < 300000]

# Отделяем метки классов от признаков
y_train = X_train['y']
X_train = X_train.drop('y', axis=1)

# Общий датасет для большего числа записей, используемых на кросс-валидации
X_general, y_general = train_df.iloc[:, :-1], train_df.iloc[:, -1]

In [ ]:
# Создание оверсемпл-датасета (отбор рандомных записей)
oversampler = RandomOverSampler(random_state=42)
X_res, y_res = oversampler.fit_resample(X_train, y_train)

## Модели

In [7]:
# Параметры модели
parameters_xgb = {
    'learning_rate': 0.23957113746389,
    'iterations': 2374,
    'max_depth': 18,
    'num_leaves': 256,
    'reg_lambda': 0.15,
    'tree_method': TREE_METHOD,
}

# Предобработчик и модель
data_preprocessor = DataPreprocessor()
model_xgb = XGBClassifier(**parameters_xgb, n_jobs=-1)

# Пайплайн
pipeline_xgb = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model_xgb)
])

In [ ]:
# Параметры модели
parameters_lgbm = {
    'objective': 'binary',
    'force_col_wise': True,
    'learning_rate': 0.25595401399911066, 
    'n_estimators': 2122, 
    'max_depth': 14, 
    'max_bin': 163, 
    'num_leaves': 176, 
    'reg_lambda': 0.15394653650963538, 
    }

# Предобработчик и модель
data_preprocessor = DataPreprocessor()
model = LGBMClassifier(**parameters_lgbm, n_jobs=-1)

# Пайплайн
pipeline_lgbm = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model)
])

In [ ]:
# Параметры
parameters_cat = {
    'cat_features': CAT_FEATURES,
    'learning_rate': 0.15239281475612,
    'n_estimators': 1873,
    'max_depth': 12,
    'l2_leaf_reg': 0.197235485663582,
    'task_type': TASK_TYPE,
}

# Предобработчик и модель
data_preprocessor = DataPreprocessor(encode_categorical=False)
model_cat = CatBoostClassifier(**parameters_cat, eval_metric='F1')

# Пайплайн
pipeline_cat = Pipeline([('data_preproc', data_preprocessor),
                         ('model', model_cat)])

## Валидация на тестовой выборке

In [ ]:
# Обучение модели и получение предсказаний для тестовой выборки
pipeline_cat.fit(X_res, y_res, model__plot=True)
preds_cat = pipeline_cat.predict_proba(X_test)

In [ ]:
# Обучение модели и получение предсказаний для тестовой выборки
pipeline_lgbm.fit(X_res, y_res)
preds_lgbm = pipeline_lgbm.predict_proba(X_test)

In [ ]:
# Обучение модели и получение предсказаний для тестовой выборки
pipeline_xgb.fit(X_res, y_res)
preds_xgb = pipeline_xgb.predict_proba(X_test)

In [ ]:
# Бленд предсказаний моделей
preds_blend = preds_xgb * 0.2 + preds_cat * 0.2 + preds_lgbm * 0.7
preds = [1 if pred[1] > 0.47 else 0 for pred in preds_blend]

In [ ]:
# Вывод результатов валидации
valid_predictions(y_test, preds, pipeline_lgbm.classes_)

## Получение предсказаний для AI Challenge

In [ ]:
# Предсказание для тестовой выборки AI Challenge
save_submission(preds, test_df)